In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import squidpy as sq
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import catboost
import json


from tqdm import tqdm
import sys
sys.path.append('/Users/rssantanu/Desktop/codebase/constrained_FM')


import argparse



In [29]:
scRNA = ad.read_h5ad(os.path.join('/Users/rssantanu/Desktop/codebase/constrained_FM', 'datasets/h5ad_processed_datasets', 'GSE232025_stereoseq.h5ad'))
X= scRNA.obsm['X_pca']
y= scRNA.obs['celltype'].to_list()

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Now y_encoded is integer labels

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(kf.split(X, y_encoded)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    # Initialize and train the model
    model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(set(y_encoded)),
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=8,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Accuracy for fold {fold}:", accuracy_score(y_test, y_pred))



/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Accuracy for fold 0: 0.8371001390820584
Accuracy for fold 1: 0.8301460361613352
Accuracy for fold 2: 0.8356807511737089
Accuracy for fold 3: 0.8269866110241697
Accuracy for fold 4: 0.8261171970092158


In [26]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y_encoded)), )
# model = catboost.CatBoostClassifier(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=8,
#     random_state=42
# )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8162378303198887
